# Исследование надёжности заёмщиков

# Содержание

- Шаг 1. Загрузим данные и подготовим их к анализу
- Шаг 2. Предобработка данных
- 2.1 Обработка пропусков
- 2.2 Замена типа данных
- 2.3 Обработка дубликатов
- 2.4 Лемматизация
- 2.5 Категоризация данных
- Шаг 3. Ответим на вопросы
- 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?
- 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?
- 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- 3.4 Как разные цели кредита влияют на его возврат в срок?
- Шаг 4. Общий вывод

# Шаг 1. Загрузим данные и подготовим их к анализу

In [1]:
import pandas as pd
import numpy as np
import pprint

from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
df_list = [data]
for df in df_list:
    print('Первые 5 строк')
    print('-'*50)
    display(df.head())
    print('-'*50)
    print('Размер датафрейма: (строк:столбцов) {}'.format(df.shape))
    print('-'*50)
    print('Типы данных и общая информация')
    print(df.info())
    print('-'*50)
    print('Наличие дубликатов: {}'.format(df.duplicated().sum()))
    print('-'*50)
    print('Наличие пропусков')
    print('-'*50)
    print(df.isna().sum())
    print('Стат данные')
    print('-'*50)
    display(df.describe())

Первые 5 строк
--------------------------------------------------


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


--------------------------------------------------
Размер датафрейма: (строк:столбцов) (21525, 12)
--------------------------------------------------
Типы данных и общая информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбцах days_employed и total_income данных меньше ,чем в других столбцах. 


В столбце days_employed много отрицательных значений.

# Шаг 2. Предобработка данных

## 2.1  Обработка пропусков

In [4]:
income_grouped_mean = data.groupby('income_type')['total_income'].mean()

In [5]:
income_grouped_median = data.groupby('income_type')['total_income'].median()

In [6]:
def fillbygroup(data, row): 
    unique_inc_type = data['income_type'].unique()  # Заменяем пропуски на медиальные значения.
    for type in unique_inc_type:
        data.loc[data['income_type'] == type, row] = data.loc[data['income_type'] == type, row].fillna(data[data['income_type'] == type]['total_income'].median())
    return data
data = fillbygroup(data, 'total_income')

In [7]:
# Чтобы заполнить пропущенные значения в столбце трудовой стаж (days_employed) в днях можно взять медиану для каждой группы возраста. Для этого создадим новый столбец "age_group"
# 1 группа: 19 - 30
# 2 группа: 30 - 40
# 3  группа: 40 - 55 
# 4  группа: 55 - 75

def days_employed(row):
    
    age = row['dob_years']

    if age <= 30:
        return '1 группа'
    
    if age <= 40 and age > 30:
        return '2 группа'
    
    if age <= 55 and age > 40:
        return '3 группа'
    
    if age <= 80 and age > 55:
        return '4 группа'

data['age_group'] = data.apply(days_employed, axis=1)

data['days_employed'] = data['days_employed'].fillna(0)
data.isnull().sum()

data.groupby('age_group')['days_employed'].mean()[0]

data.loc[(data['age_group'] == '1 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[0]
data.loc[(data['age_group'] == '2 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[1]
data.loc[(data['age_group'] == '3 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[2]
data.loc[(data['age_group'] == '4 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[3]

data[data['total_income'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

Пропуски в столбцах "общий трудовой стаж" и "средний доход", я предполагаю, появились по причине того, что ( в основном он пропущен у возрастных людей) они жили в другой стране ( СССР ), их документы могут быть утрачены. Также могу предположить халатность сотрудников при приёме документов на кредитование и при внесение этой информации в базу данных. 


Данные среднего и медианы по некоторым столбцам совпадают, по другим отличаются незначительно. 


Использую для заполнения пропусков медиальные значения.

## 2.2  Замена типа данных

In [8]:
data['total_income'] = data['total_income'].round().astype('int') #  округляю значения в большую сторону и перевожу в 'int'
data['days_employed'] = data['days_employed'].round().astype('int')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
 12  age_group         21525 non-null  object
dtypes: int32(2), int64(5), object(6)
memory usage: 2.0+ MB


Выбран метод DataFrame.astype() как наиболее удобный, для замены типа данных

## 2.3  Обработка дубликатов

In [10]:
print(data.duplicated().sum())
print(data['education'].unique())
data['education'] = data['education'].replace('ВЫСШЕЕ', 'высшее')   # переименовываем в однородные названия
data['education'] = data['education'].replace('Высшее', 'высшее')
data['education'] = data['education'].replace('СРЕДНЕЕ', 'среднее')
data['education'] = data['education'].replace('Среднее', 'среднее')
data['education'] = data['education'].replace('НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'неоконченное высшее')
data['education'] = data['education'].replace('Неоконченное высшее', 'неоконченное высшее')
data['education'] = data['education'].replace('НАЧАЛЬНОЕ', 'начальное')
data['education'] = data['education'].replace('Начальное', 'начальное')
data['education'] = data['education'].replace('УЧЕНАЯ СТЕПЕНЬ', 'ученая степень')
data['education'] = data['education'].replace('Ученая степень', 'ученая степень')
data = data.drop_duplicates().reset_index()    # убираем дубликаты
print(data.duplicated().sum())

54
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
0


## 2.4  Лемматизация

In [11]:
# Узнаем количество уникальных значений — 38
unique_credit_purposes = df['purpose'].unique().tolist()
unique_credit_purposes

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [12]:
m = Mystem()
string = '; '.join(unique_credit_purposes)
pprint.pprint(string)

('покупка жилья; приобретение автомобиля; дополнительное образование; сыграть '
 'свадьбу; операции с жильем; образование; на проведение свадьбы; покупка '
 'жилья для семьи; покупка недвижимости; покупка коммерческой недвижимости; '
 'покупка жилой недвижимости; строительство собственной недвижимости; '
 'недвижимость; строительство недвижимости; на покупку подержанного '
 'автомобиля; на покупку своего автомобиля; операции с коммерческой '
 'недвижимостью; строительство жилой недвижимости; жилье; операции со своей '
 'недвижимостью; автомобили; заняться образованием; сделка с подержанным '
 'автомобилем; получение образования; автомобиль; свадьба; получение '
 'дополнительного образования; покупка своего жилья; операции с недвижимостью; '
 'получение высшего образования; свой автомобиль; сделка с автомобилем; '
 'профильное образование; высшее образование; покупка жилья для сдачи; на '
 'покупку автомобиля; ремонт жилью; заняться высшим образованием')


In [13]:
lemmas = m.lemmatize(string)
pprint.pprint(Counter(lemmas))

Counter({' ': 59,
         '; ': 37,
         'покупка': 10,
         'недвижимость': 10,
         'автомобиль': 9,
         'образование': 9,
         'жилье': 7,
         'с': 5,
         'операция': 4,
         'на': 4,
         'свой': 4,
         'свадьба': 3,
         'строительство': 3,
         'получение': 3,
         'высокий': 3,
         'дополнительный': 2,
         'для': 2,
         'коммерческий': 2,
         'жилой': 2,
         'подержать': 2,
         'заниматься': 2,
         'сделка': 2,
         'приобретение': 1,
         'сыграть': 1,
         'проведение': 1,
         'семья': 1,
         'собственный': 1,
         'со': 1,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})


После группировки цели кредита разделены на 4 группы (автомобиль, образование, свадьба, недвижимость)

## 2.5  Категоризация данных

In [14]:
# Сгрупируем данные о доходе в месяц 
# 1 группа: 15 - 55
# 2 группа: 55 - 120
# 3  группа: 120 - 200
# 4  группа: 200 - 2000000

def income_group(row):
    
    income = row['total_income']

    if income <= 55000:
        return 'Доход 15-55'
    
    if income <= 120000 and income > 55000:
        return 'Доход 55-20'
    
    if income <= 200000 and income > 120000:
        return 'Доход 120-200'
    
    if income <= 4000000 and income > 200000:
        return 'Доход > 200'

data['income_gr'] = data.apply(income_group, axis=1)

#print(data['children'].value_counts())
data['children'] = data['children'].apply(abs) # убираю отрицательные значения по количеству детей.
data['days_employed'] = data['days_employed'].apply(abs) # Убираем отрицательные значения. ( в трудовом стаже их быть не может)

Отрицательные значения могли появиться, как ошибки при вводе данных.

# Шаг 3. Ответим на вопросы

## 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
number_of_children_set = df.groupby('children')['debt'].sum() / \
    df.groupby('children')['debt'].count() * 100

In [16]:
number_of_children_set

children
-1      2.127660
 0      7.512898
 1      9.215442
 2      9.440389
 3      8.181818
 4      9.756098
 5      0.000000
 20    10.526316
Name: debt, dtype: float64

In [17]:
grouped_number_of_children = df.groupby('children')['debt'].count()
grouped_number_of_children

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64

In [18]:
children_pivot = df.pivot_table(index = ['children'], \
                                columns = ['debt'], \
                                values = 'purpose', aggfunc='count')

children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
children_pivot

debt,0,1,ratio
children,,,
-1,46.0,1.0,2.173913
0,13086.0,1063.0,8.123185
1,4374.0,444.0,10.150892
2,1861.0,194.0,10.424503
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN
20,68.0,8.0,11.764706


Можно отметить, что люди с 5 детьми показывают 0, но выборка по ним не репрезентативна. 

Бездетные отдают кредиты хуже.

## 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
data = data.replace('гражданский брак', 'женат / замужем')
data.reset_index(drop=True)
print(data['family_status'].value_counts())
fam_conv = (data[data['debt'] == 1].groupby('family_status')['debt'].count() / data.groupby('family_status')['debt'].count()).round(3)
fam_conv.to_frame('conversion table')

женат / замужем          16490
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


,conversion table
family_status,
Не женат / не замужем,0.098
в разводе,0.071
вдовец / вдова,0.066
женат / замужем,0.080


Несемейные люди возвращают кредиты хуже всех. 

Семейные люди лучше планируют свой бюджет. 

Интересные результаты по людям, побывавшим в браке, они оказываются самыми ответственными кредиторами

## 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
data_pivot3 = data.pivot_table(index=['debt'], columns='income_gr', values='age_group', aggfunc='count')

in1 = data_pivot3['Доход 120-200'][1] / data_pivot3['Доход 120-200'][0]
in2 = data_pivot3['Доход 15-55'][1] / data_pivot3['Доход 15-55'][0]
in3 = data_pivot3['Доход 55-20'][1] / data_pivot3['Доход 55-20'][0]
in4 = data_pivot3['Доход > 200'][1] / data_pivot3['Доход > 200'][0]

print("{0:.2f}% Доход 120-200".format(in1*100))
print("{0:.2f}% Доход 15-55".format(in2*100))
print("{0:.2f}% Доход 55-20".format(in3*100))
print("{0:.2f}% Доход > 200".format(in4*100))

9.53% Доход 120-200
6.42% Доход 15-55
9.03% Доход 55-20
7.60% Доход > 200


Люди с небольшим доходом, максимально ответственно относятся к своим кридитам, а так же люди с большим доходом.

## 3.4 Как разные цели кредита влияют на его возврат в срок?

In [22]:
# Попробуем через map(), составим словарь из двух списков
credit_purpose_keys= []
credit_purpose_list = list(df.purpose.unique())
for value in range(len(credit_purpose_list)):
    if 'образован' in credit_purpose_list[value]:
        credit_purpose_keys.append('Образование')
    elif 'авто' in credit_purpose_list[value]:
        credit_purpose_keys.append('Автомобиль')
    elif 'свадь' in credit_purpose_list[value]:
        credit_purpose_keys.append('Свадьба')
    else:
        credit_purpose_keys.append('Недвижимость')
        
credit_purpose_keys

['Недвижимость',
 'Автомобиль',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Автомобиль',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Свадьба',
 'Образование',
 'Недвижимость',
 'Недвижимость',
 'Образование',
 'Автомобиль',
 'Автомобиль',
 'Образование',
 'Образование',
 'Недвижимость',
 'Автомобиль',
 'Недвижимость',
 'Образование']

In [23]:
credit_purpose_dict = dict(zip(credit_purpose_list, credit_purpose_keys))

In [25]:
df['credit_purpose_status'] = df['purpose'].map(credit_purpose_dict)

In [28]:
credit_purpose_pivot = df.pivot_table(index=['credit_purpose_status'], columns=['debt'], values='education_id', aggfunc='count')
credit_purpose_pivot['ratio'] = credit_purpose_pivot[1] / credit_purpose_pivot[0] * 100
credit_purpose_pivot

debt,0,1,ratio
credit_purpose_status,,,
Автомобиль,3912,403,10.301636
Недвижимость,10058,782,7.774906
Образование,3652,370,10.131435
Свадьба,2162,186,8.603145


Хуже всего выглядят свадьбы и недвижимость. Выглядит логично, ипотека — пожалуй, самая крупная кредитная и рисковая сделка для частного лица. Свадьбы — это не инвестиционный проект и это неликвидный кредит, который по сути проедают.

# Шаг 4. Общий вывод

При работе с данными были заполнены пропущенные значения, удалены дубликаты и выделены категории для целей кредита, группы по возрасту. 

Применив сводные таблицы, я оценил, какие критерии влияют на возврат кредита. 

Дополнительный анализ может помочь более точно предсказать вероятность выплаты кредита в срок и подготовить предложения по кредитованию для точечных групп клиентов.